In [2]:
from lxml import html, etree
import requests
import pandas as pd
import regex

In [92]:
def get_source_code(url):
    loaded_data = requests.get(url)
    return loaded_data.text

In [160]:
def extract_headlines(url, news_list):    
    text = get_source_code(url)
    tree = html.fromstring(text)
    headlines = tree.xpath('.//div[@class = "searchresult"]/a/text()')
    dates = tree.xpath('.//div[@class="deemphasized"]/span/text()')    
    
    for headline, date in zip(headlines, dates):
        news_list.append({
            'date' : date,
            'headline' : headline
        })
        
    return len(headlines)

In [161]:
def load_news(url_first_page):
    news_list = []
    extract_headlines(url=url_first_page, news_list=news_list)
    print('Loaded headlines: ', len(news_list), flush=True)
    
    num = 1
    while True:
        current_page_url = url_first_page + '&o={}'.format(num * 100 + 1)
        headlines_num = extract_headlines(url=current_page_url, news_list=news_list)
        num = num + 1
        if(headlines_num == 0):
            break
        print('Loaded headlines: ', len(news_list), flush=True)
        
    return pd.DataFrame(news_list)

In [164]:
def load_news_by_tegs_from_marketwatch(tegs_list):
    news_by_tegs_dict = {}
    for teg in tegs_list:
        teg_url = 'https://www.marketwatch.com/search?q={}&m=Keyword&rpp=100&mp=159&bd=false&rs=true'.format(teg)
        print('----Loading for', teg)
        teg_pandas_dataframe = load_news(url_first_page=teg_url)
        news_by_tegs_dict[teg] = teg_pandas_dataframe
        
    print('-------Loading finished-------')
        
    return news_by_tegs_dict

In [163]:
tegs = ['microsoft', 'apple']
news_dict = load_news_by_tegs_from_marketwatch(tegs_list=tegs)

Loading for microsoft
Loaded headlines:  100
Loaded headlines:  200
Loaded headlines:  300
Loaded headlines:  400
Loaded headlines:  500
Loaded headlines:  600
Loaded headlines:  700
Loaded headlines:  800
Loaded headlines:  900
Loaded headlines:  1000
Loaded headlines:  1100
Loaded headlines:  1200
Loaded headlines:  1300
Loaded headlines:  1400
Loaded headlines:  1500
Loaded headlines:  1600
Loaded headlines:  1700
Loaded headlines:  1800
Loaded headlines:  1900
Loaded headlines:  2000
Loaded headlines:  2100
Loaded headlines:  2200
Loaded headlines:  2300
Loaded headlines:  2400
Loaded headlines:  2500
Loaded headlines:  2600
Loaded headlines:  2617
Loading for apple
Loaded headlines:  100
Loaded headlines:  200
Loaded headlines:  300
Loaded headlines:  400
Loaded headlines:  500
Loaded headlines:  600
Loaded headlines:  700
Loaded headlines:  800
Loaded headlines:  900
Loaded headlines:  1000
Loaded headlines:  1100
Loaded headlines:  1200
Loaded headlines:  1300
Loaded headlines: 

In [168]:
news_dict['apple']

,date,headline
0,"6:45 p.m. Sept. 16, 2020",Global Computing Mouse Market- Featuring A4TEC...
1,"6:20 p.m. Sept. 16, 2020",U.S. Cellular to Offer the Breakthrough Apple ...
2,"9:45 a.m. Sept. 16, 2020",Global Audio and Video Editing Software Market...
3,"9:25 a.m. Sept. 16, 2020",Apple Unveils New Watches and iPads at Live “T...
4,"9:00 a.m. Sept. 16, 2020",Anthem Announces Research Study with Apple Wat...
...,...,...
2207,"5:00 a.m. May 15, 2013",QUAN Prepares New Product for $2.3 Billion iPa...
2208,"2:32 p.m. May 14, 2013","Apple REIT Six, Inc. Announces Closing of Merger"
2209,"7:00 a.m. May 14, 2013",Apple Federal Credit Union Completes Merger wi...
2210,"12:17 p.m. May 13, 2013",Market Strategies International Survey: Apple ...
